In [ ]:
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install seaborn
# !pip install scikit-learn

# !pip list

Importing Libraries

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tqdm import tqdm

Loading dataset into dataframe

In [5]:
# Load dataset
file_path = "C:\\Users\\HP\Desktop\\py test\\resume_data.csv"  # Update path if needed
df = pd.read_csv(file_path)

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\HP\AppData\Local\Temp\ipykernel_16164\3003074318.py:2: SyntaxWarning: invalid escape sequence '\D'
  file_path = "C:\\Users\\HP\Desktop\\py test\\resume_data.csv"  # Update path if needed


Data Pre-Processing

In [7]:
# Fill missing values and combine text fields for resume and job
df['resume_text'] = (
    df['skills'].fillna('') + ' ' +
    df['career_objective'].fillna('') + ' ' +
    df['responsibilities'].fillna('')
)

In [9]:
df['job_text'] = (
    df['skills_required'].fillna('') + ' ' +
    df['responsibilities.1'].fillna('') + ' ' +
    df['educationaL_requirements'].fillna('')
)

TF-IDF Vectorization

In [11]:
# TF-IDF Vectorization
vectorizer = TfidfVectorizer(stop_words='english')
resume_vectors = vectorizer.fit_transform(df['resume_text'])
job_vectors = vectorizer.transform(df['job_text'])

Cosine similarity-based matching

In [13]:
# Compute cosine similarity row-wise
match_scores = []
for i in tqdm(range(len(df))):
    resume_vec = resume_vectors[i]
    job_vec = job_vectors[i]
    sim = cosine_similarity(resume_vec, job_vec)[0][0]
    match_scores.append(sim)

df['computed_match_score'] = match_scores

100%|██████████| 9544/9544 [00:07<00:00, 1231.62it/s]


In [15]:
# Save sample of match scores
print("\nSample of Matching Scores:")
print(df[['job_position_name', 'computed_match_score', 'matched_score']].head())


Sample of Matching Scores:
                                   job_position_name  computed_match_score  \
0                           Senior Software Engineer              0.213016   
1                     Machine Learning (ML) Engineer              0.724611   
2  Executive/ Senior Executive- Trade Marketing, ...              0.620561   
3                     Business Development Executive              0.310939   
4                                Senior iOS Engineer              0.338474   

   matched_score  
0       0.850000  
1       0.750000  
2       0.416667  
3       0.760000  
4       0.650000  


Classification steps is followed

In [17]:
# Classification: Predict job_position_name from resume_text
X = resume_vectors
y = df['job_position_name']

In [19]:
# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [21]:
# Train Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

Evaluation

In [22]:
report = classification_report(y_test, y_pred, zero_division=0)
print("\nClassification Report:")
print(report)


Classification Report:
                                                                                         precision    recall  f1-score   support

                                                                            AI Engineer       1.00      1.00      1.00        68
                                                 Asst. Manager/ Manger (Administrative)       1.00      1.00      1.00        68
                                                         Business Development Executive       1.00      1.00      1.00        68
                                                                         Civil Engineer       1.00      1.00      1.00        69
                                                                          Data Engineer       1.00      1.00      1.00        68
                                                                  Data Science Engineer       1.00      1.00      1.00        68
                                                           Database Admi

In [27]:
#save result in csv or can view in console
df.to_csv("resume_matching_results.csv", index=False)
# df